## Övning 5
__John Landeholt__

johnlan@kth.se


__agenda:__

* Kattis (Lab 8)
    * Testning
* Syntax (Lab 9)
* Automater (Föreläsning 10)
    * KMP-automat

### Kattis

<img src="img/ö5i1.png" width="33%" style="float:right;"/>

Kattis är en plattform som kontrollerar ditt programs `korrekthet`. Den gör det igenom att köra ditt program på olika testfall, och om den får ett resultat den förväntar sig, så är programmet korrekt utfört, annars inte.

I __labb 9__ ska vi använda oss utav [Formelkoll](https://kth.kattis.com/problems/kth.tilda.formelkoll2) som kommer att läsa en molekylformel med hjälp av ditt program och sedan jämföra det med sin databas för att se om det stämmer.

>__notera__: Kattis är verkligen inte perfekt, pedagogiskt eller lätt. Hon kommer endast berätta att det är fel och inte varför det är fel. Det är upp dit datalogen att förstå vad som kan vara fel.

Hjälpkod för att läsa och skriva till Kattis:

[Kattio](https://github.com/landeholt/tilda/blob/main/help-code/kattio.py)

[Skelettkod till lab 8 och 9](https://github.com/landeholt/tilda/blob/main/skeleton-code/molecule.py)

In [ ]:
# Kattio
from sys import stdin, stdout

def get_formula():
    return stdin.readline().strip()

def get_formulas():
    for line in stdin:
        yield line.strip()

def send(out):
    stdout.write(str(out) + '\n')

### Testning

Innan man skickar in till __kattis__ kan det vara bra och faktiskt testa sin kod lokalt först. Detta gör man genom att skriva testfiler som kontrollerar om inputen resulterar till den förväntade outputen.

[testfil till lab 8 och 9](https://github.com/landeholt/tilda/blob/main/help-code/test_lab8.py)

In [ ]:
from YOUR_FILE import YOUR_FUNCTION as f
# from lab8 import formula as f
import pytest

@pytest.mark.parametrize('formula, expected', [
    ('Na', 'Formeln är syntaktiskt korrekt'),
    ('H2O', 'Formeln är syntaktiskt korrekt'),
    ('Si(C3(COOH)2)4(H2O)7', 'Formeln är syntaktiskt korrekt'),
    ('Na332', 'Formeln är syntaktiskt korrekt'),
    ('C(Xx4)5', 'Okänd atom vid radslutet 4)5'),
    ('C(OH4)C', 'Saknad siffra vid radslutet C'),
    ('C(OH4C', 'Saknad högerparentes vid radslutet'),
    ('H2O)Fe', 'Felaktig gruppstart vid radslutet )Fe'),
    ('H0', 'För litet tal vid radslutet'),
    ('H1C', 'För litet tal vid radslutet C'),
    ('H02C', 'För litet tal vid radslutet 2C'),
    ('Nacl', 'Saknad stor bokstav vid radslutet cl'),
    ('a', 'Saknad stor bokstav vid radslutet a'),
    ('(Cl)2)3', 'Felaktig gruppstart vid radslutet )3'),
    (')', 'Felaktig gruppstart vid radslutet )'),
    ('2', 'Felaktig gruppstart vid radslutet 2'),
    ('', 'Formeln är syntaktiskt korrekt'),
    ('H111', 'Formeln är syntaktiskt korrekt'),
    ('H110', 'Formeln är syntaktiskt korrekt')
])

def test_formulas(formula, expected):
    cert = f(formula)
    assert cert == expected

In [7]:
!pytest test_lab8.py

============================= test session starts ==============================
platform darwin -- Python 3.9.0, pytest-6.2.1, py-1.10.0, pluggy-0.13.1
rootdir: /Users/John/Documents/KTH/projects/stora-boken-om-nada
collected 19 items                                                             

test_lab8.py ...................                                         [100%]

============================== 19 passed in 0.04s ==============================


In [8]:
!pytest test_lab8_wrong.py

============================= test session starts ==============================
platform darwin -- Python 3.9.0, pytest-6.2.1, py-1.10.0, pluggy-0.13.1
rootdir: /Users/John/Documents/KTH/projects/stora-boken-om-nada
collected 19 items                                                             

test_lab8_wrong.py ....F.F.F.F..FF.FF.                                   [100%]

=================================== FAILURES ===================================
___________ test_formulas[C(Xx4)5-Ok\xe4nd atom vid rafdslutet 4)5] ____________

formula = 'C(Xx4)5', expected = 'Okänd atom vid rafdslutet 4)5'

    @pytest.mark.parametrize('formula, expected', [
        ('Na', 'Formeln är syntaktiskt korrekt'),
        ('H2O', 'Formeln är syntaktiskt korrekt'),
        ('Si(C3(COOH)2)4(H2O)7', 'Formeln är syntaktiskt korrekt'),
        ('Na332', 'Formeln är syntaktiskt korrekt'),
        ('C(Xx4)5', 'Okänd atom vid rafdslutet 4)5'),
        ('C(OH4)C', 'Saknad siffra vid radslutet C'),
        ('C

### Syntax

__vad är syntax?__

Det är en uppsättning regler för vilken `ordning` något får ske. När `python` läser in ditt program och analyserar det innan körningen så sker det en syntaktiskt kontroll för att säkertställa att programmet är `giltigt`. Men bara för att något är giltigt, betyder det inte att det är `korrekt` eller för den delen `ändligt`.

En syntax kan vara rekursiv.

    <mol> ::= <group> | <group><mol>
    <group> ::= <atom> |<atom><num> | (<mol>) <num>
    ...
    
Vilket i det här fallet innebär att en molekyl kan vara __en__ eller __flera__ grupper.

### Rekursiv medåkning

__vad menas med medåkning?__

Vi kallar något medåkning när en funktion anropar __en__ eller __flera__ andra funktioner `ärver` in-parametrarna från tidigare funktionsanrop.

> Eftersom objekt i python är `mutable`, så innebär detta att ett objekt kan förändras inuti funktioner, där vi tidigare kanske antagigt att endast lokala förändringar görs.

__List exempel__:

In [9]:
def write_to_list(_list):
    if len(_list) < 5:
        index = len(_list)
        _list.append(index)
        write_to_list(_list)

def remove_from_list(_list):
    item = 3
    if item in _list:
        _list.remove(item)

def read_from_list(_list):
    for item in _list:
        print(item, end=' ')
    print()

index_list = []
read_from_list(index_list)
write_to_list(index_list)
read_from_list(index_list)
remove_from_list(index_list)
read_from_list(index_list)


0 1 2 3 4 
0 1 2 4 


### Rekursiv medåkning

__är detta ett problem?__

Det beror helt på situationen. I lab 9 kommer det vara tvunget att använda sig utav rekursiv medåkning för att kontrollera syntaxen. Detta görs enklast med en datastruktur som är `länkad`.

* Kö - en metod .peek() behöver läggas till
* dubbellänkad lista
* Stack - rekommenderar __inte__, tänk efter!

Och som tidigare exempel visade, så kan objekt förändras inuti funktioner, även om de inte returnerar något. Detta leder till att vi måste säkerställa att inget lämnar datastrukturen när det inte ska göra det. Dvs, det får bara lämna `datastrukturen` när det är `syntaktiskt korrekt`

### Syntax

__Hur skriver man en syntax?__

För att kunna skriva en syntax behöver vi veta dens `environment` och `beteende`.

Vi kan ta ett exempel från tentamen __2000-08-31__:

Vi ska koda en webbläsare och behöver nu implementera en renderings-funktion till webbläsaren!

Som test använder vi oss utav denna filen index.qhtml:

    Organismer <BR> <Q> Djur <BR> <Q> Flugor <BR> Sillar <BR> </Q> Svamp <BR> <Q> Flugsvamp <BR>
    Sillkremla <BR> </Q> </Q>
   
Där vi måste greja biffen att se till att vår renderings-funktion förstår vad de olika egenskaperna betyder för vår rendering!

Vi läser på om taggarna till .qhtml-filen och ser att `<Q> </Q>` innebär att element innanför ska vara indraget, samt att `<BR>` innebär en ny rad.

Men innan vi bygger renderings-funktionen så vill vår chef att vi tar fram en `syntax` __för__ webbfilen

Vi börjar med att analysera vilka regler som får finnas i webbfilen och märker att det måste vara:
* `<text>` - vanlig text som inte är inuti några taggar
* `<BR>`
* `<Q>`
* `</Q>`

Men också `<webbfil>`, eftersom det är __för__ den som vi gör syntaxen till. Hur annars ska vi få tillgång till innehållet?

`<webbfil>` får innehålla alla ovanstående, så vi börjar där!

    <webbfil> ::= <text> | <webbfil><BR><webbfil> | <Q><webbfil></Q>

__Varför?__

Vi säger att `<webbfil>` får vara:
* text 
* [text, eller annat]`<BR>`[text, eller annat]
* `<Q>`[text, eller annat]`</Q>`

Eftersom webbfil är en rekursion.

Sist behöver vi förklara vad de olika taggarna får vara för något:

    <text> ::= A | .. | Ö <text>
    <BR>   ::= "<BR>"
    <Q>    ::= "<Q>"
    </Q>   ::= "</Q>"

Där `<text>` också har en rekursion som tillåter alfabetiska strängar. Såklart kan siffror och tecken läggas till i efterhand, men för vårt webbfil __index.qhtml__ behövs ingen annat än alfabetet.

### Automater

__vad är en automat?__

Det är en maskin, eller i vårt fall en kodbas som har olika `tillstånd` (state). När man matar automaten med `indata` kommer automatens `tillstånd` eventuellt förändras beroende på om indatan ger upphov till det.

<img src="img/ö5i2.png" style="float:right;"/>

Tanken är att `indatan` används av maskinen och skapar ett nytt bestämt `tillstånd` utav den indatan, som man kan se i bilden till höger. Där inputen är Y, vilket leder till att man hamnar i state C.




__Exempel på automat__:

<img src="https://miro.medium.com/max/1920/1*i83QledYJW6nHqtqlmXwIw.gif" style="float:right;width:10%"/>

Ett trafikljus är en automat. Den har 4 `tillstånd`, där varje `tillstånd` har en unik lampa igång, om inte avstängd.Den har även en given följd, där om tillståndet är __grön__, så måste nästa input vara __röd__, annars har ett fel uppstått som gör att automaten sätts i en `evighetsloop` där `tillståndet` är __gul__


In [10]:
# Traffic light example as a automata
from time import sleep
from shared import Automata

def get_state(automata,_input):
    states = {
        'on': 'red',
        'green': 'red',
        'yellow': 'green',
        'red': 'yellow',
    }
    try:
        automata.state = states[_input]
    except:
        automata.state = 'fail_state'
    finally:
        return automata.state
    
def run(automata, _input):
    for _ in range(3):
        _input = get_state(automata, _input)
        print(automata)
        sleep(0.5)

automata = Automata()

run(automata, 'on')
run(automata, None)

RED
YELLOW
GREEN
YELLOW
YELLOW
YELLOW


__ett mer datalognära exempel__:

<img src="img/kth-kmp.png" style="float:right" />

Textsökning är också en automat, som får en input som är det som ska sökas efter. Därefter går automaten igenom texten bokstav för bokstav och letar efter den följd av bokstäver som givits som input. I detta fall är det varje bokstav som är automatens `tillstånd`.

En känd textsökningsalgoritm är KMP-Algoritmen. Det är __viktigt__ att __notera__ att __KTH's version__ av KMP automaten __skiljer__ sig extremt mycket från resterande världens KMP automat!!

__Generella regler för KTH-KMP:__

* Om s[0], alltså första tecknet i söksträngen förekommer någonstans i strängen, så får det tolkas som ett prefix.
* Välj största möjliga prefixet
* Om flera olika prefixer är inuti varandra, så ignoreras det prefixet som annars hade genererat en siffra.
* Om ett index i söksträngen kan ge upphov till flera pekar-index, så ska man välja det högsta indexet.

__2019-03-13 [1]__

<img src="img/kth-kmp-2.png" style="float:right"/>

Vi börjar enkelt för att pröva de tidigare nämda reglerna! Vi ser att det finns ett prefix i söksträngen. __KÖ__. Så efter alla förekomster förutom första gången så placerar vi index-siffran efter prefixet. I detta fallet är det (3, A).

Därefter börjar vi från vänster och arbetar oss till höger. `n[0]` ska alltid vara __0__, sedan om det tillhör ett prefix eller inget alls så är det __1__. Om `s[0]` förekommer och inte har ett annat prefix framför sig, så sätt den som __0__.

__2019-10-21 [1]__

<img src="img/kth-kmp-1.png" style="float:right"/>

Här ser vi att inga prefix finns, men det finns gott om förekomster av `s[0]`. Vi upprepar samma regler som tidigare och börjar med att placera ut värden för vår nextvektor för de vi redan vet om. Som i det här fallet är `n[i]` för platserna bakom alla förekomster av `s[0]`

__2017-10-20 [1]__

<img src="img/kth-kmp-3.png" style="float:right"/>

Nu börjar det bli lite knepigare! Funkar våra regler här också? Vi prövar! Här hittar vi 2 olika prefix. __PAP__ och __PA__. Sedan märker vi av att det finns flera förekomster av `s[0]`.

Vi börjar med att kryssa av prefix-positionerna och börjar från höger. Där vi ser att `n[8]` blir __3__, eftersom __PA__ förekommer före `s[8]`. Därefter ser vi att `n[7]`blir 4, för att __PAP__ förekommer före `s[7]`. Men varför inte __2__, då det också är en förekomst av `s[0]`? Det är för att man alltid väljer det högsta indexet möjligt.

Vid `n[6]` kommer en luring! __PA__ förekommer före `s[6]`, men eftersom `s[6]` är en del av ett annat prefix, så ignorerars det! Sist, men inte minst så finns det en förekomst av `s[0]` som inte berors av ett prefix, vilket är `n[4]`, så vi sätter den till __2__.